## Resource-Constrained Project Scheduling Problem with Transfer Times

This notebook demonstrates how to model and solve the Resource-Constrained Project Scheduling Problem with Sequence-Dependent Setup Times
using Constraint Programming with IBM’s CP Optimizer via the [`docplex.cp`](https://ibmdecisionoptimization.github.io/docplex-doc/cp/refman.html) Python API.
This problem extends the classical RCPSP (see [`rcpsp.ipynb`](https://github.com/radovluk/CP_Cookbook/blob/main/notebooks/rcpsp.ipynb)).

### Problem Definiton

The Resource-Constrained Project Scheduling Problem with Trasfer Times (RCPSPTT) is an extension of the RCPSP problem defined. In RCPSP, activities are executed using available resources, precedence relationships between them must be adhered to, and the total makespan is minimized. Compared to the RCPSP, RCPSPTT introduces transfer times between activities. This means that resources do not have to be directly available after the activity that uses them finishes; rather, a transfer of resources to the following activity might be required. The transfer time depends on both the activities and the resource transferred.

Formally, the RCPSPTT problem is defined as follows:

Let $\mathcal{A}=\{0,1,\ldots,\hat{A}\}$, $\ \hat{A}\in\mathbb{N}$, be a set of activities with indices $i,j\in\mathcal{A}$.

Let $P\in\mathbb{N}^{|\mathcal{A}|}$ be a vector of activity processing times.

Let $\mathcal{E}=\{(i,j)\mid \text{activity } i \text{ precedes activity } j\}$ be a set of precedences.

Let $\mathcal{R}=\{0,1,\ldots,\hat{R}\}$, $\ \hat{R}\in\mathbb{N}$, be a set of primary resources indexed by $r\in\mathcal{R}$.

Let $C\in\mathbb{N}^{|\mathcal{R}|}$ be a vector of primary resource capacities.

Let $\mathbf{Q}\in\mathbb{Z}_{{\ge 0}}^{|\mathcal{A}|\times|\mathcal{R}|}$ be the activity–resource consumption matrix, with entries $\mathbf{Q}_{i,r}$ denoting the consumption of resource $r$ by activity $i$ ($i=0 \Rightarrow \mathbf{Q}_{i,r}=C_r$).

Let $\Delta\in\mathbb{Z}_{\ge 0}^{|\mathcal{A}|\times|\mathcal{A}|\times|\mathcal{R}|}$ be a matrix of transfer delays, with entries $\Delta_{i,j,r}$ denoting the delay required if resource $r$ is transferred between activities $i$ and $j$.

### CPLEX Formulation

$$
\begin{aligned}
\min\quad
& \mathrm{end}(a_{\hat{A}})
& &
& \text{(1)} \\[8pt]
\text{s.t.}\quad
& \mathrm{endBeforeStart}(a_i,\ a_j)
& & \forall (i,j)\in\mathcal{E}
& \text{(2)} \\[6pt]
& \sum_{j\in\mathcal{A}} f_{0,j,r} \;=\; C_r
& & \forall r\in\mathcal{R}
& \text{(3)} \\[10pt]
& f_{i,j,r}\ \ge\ 1 \ \implies\ \mathrm{presenceOf}\!\left(z_{i,j,r}\right)=1
& & \forall i,j\in\mathcal{A},\ \forall r\in\mathcal{R}:\ \Delta_{i,j,r}=0
& \text{(4)} \\[12pt]
& \sum_{j\in\mathcal{A}:\ \Delta_{j,i,r}>0} \mathrm{heightAtStart}\!\Bigl(z_{j,i,r},\ \mathrm{pulse}\!\bigl(z_{j,i,r},(0,\mathbf{Q}_{j,r})\bigr)\Bigr)
\;+\;
\sum_{j\in\mathcal{A}:\ \Delta_{j,i,r}=0} f_{j,i,r}
\;=\; \mathbf{Q}_{i,r}
& & \forall i\in\mathcal{A}\setminus\{0,\hat{A}\},\ \forall r\in\mathcal{R}
& \text{(5)} \\[12pt]
& \sum_{j\in\mathcal{A}:\ \Delta_{i,j,r}>0} \mathrm{heightAtStart}\!\Bigl(z_{i,j,r},\ \mathrm{pulse}\!\bigl(z_{i,j,r},(0,\mathbf{Q}_{i,r})\bigr)\Bigr)
\;+\;
\sum_{j\in\mathcal{A}:\ \Delta_{i,j,r}=0} f_{i,j,r}
\;=\; \mathbf{Q}_{i,r}
& & \forall i\in\mathcal{A}\setminus\{0,\hat{A}\},\ \forall r\in\mathcal{R}
& \text{(6)} \\[12pt]
& \mathrm{endBeforeStart}(a_i,\ z_{i,j,r}),
\quad
\mathrm{endBeforeStart}(z_{i,j,r},\ a_j)
& & \forall i,j\in\mathcal{A},\ \forall r\in\mathcal{R}:\ \Delta_{i,j,r}>0
& \text{(7)} \\[10pt]
& \sum_{i\in\mathcal{A}} \mathrm{pulse}(a_i,\ \mathbf{Q}_{i,r})
\;+\;
\sum_{i,j\in\mathcal{A}:\ \Delta_{i,j,r}>0} \mathrm{pulse}\!\bigl(z_{i,j,r},\ (0,\mathbf{Q}_{i,r})\bigr)
\ \le\ C_r
& & \forall r\in\mathcal{R}
& \text{(8)} \\[14pt]
& \text{interval } a_i,\ \text{size} = P_i
& & \forall i\in\mathcal{A}
& \text{(9a)} \\
& \text{interval } z_{i,j,r}\ \text{(optional), size} = \Delta_{i,j,r}
& & \forall i,j\in\mathcal{A},\ \forall r\in\mathcal{R}:\ \Delta_{i,j,r}>0
& \text{(9b)} \\
& f_{i,j,r}\in \mathbb{Z}_{\ge 0}
& & \forall i,j\in\mathcal{A},\ \forall r\in\mathcal{R}:\ \Delta_{i,j,r}=0
& \text{(9c)}
\end{aligned}
$$


**Objective:**
- **(1)** the project makespan — the end time of the finish activity $a_{\hat{A}}$ — is minimized.

**Modeling constraints:**
- **(2)** precedence: for every $(i,j)\in\mathcal{E}$, enforce $\mathrm{endBeforeStart}(a_i,a_j)$; activity $j$ can start only after $i$ ends.
- **(3)** source capacity release: the dummy source $0$ emits exactly the available units of each primary resource: $\sum_{j\in\mathcal{A}} f_{0,j,r}=C_r$ for all $r\in\mathcal{R}$.
- **(4)** transfer activation ($\Delta=0$): if any flow is routed from $i$ to $j$ for resource $r$, the corresponding transfer interval must be present: $f_{i,j,r}\ge 1 \Rightarrow \mathrm{presenceOf}(z_{i,j,r})=1$. **There is probably a contradiciton flaw in (4)!!!**
- **(5)** inflow balance at each activity $i\in\mathcal{A}\setminus\{0,\hat{A}\}$: the amount of resource $r$ that arrives at the start of $i$ equals its consumption $\mathbf{Q}_{i,r}$.
- **(6)** outflow balance from each activity $i\in\mathcal{A}\setminus\{0,\hat{A}\}$: exactly $\mathbf{Q}_{i,r}$ units must be sent onward after $i$ finishes.
- **(7)** temporal linking of transfers ($\Delta>0$): an active transfer interval $z_{i,j,r}$ starts after $a_i$ finishes and ends before $a_j$ starts:
  $\mathrm{endBeforeStart}(a_i, z_{i,j,r}),\ \mathrm{endBeforeStart}(z_{i,j,r}, a_j).$
  since $\mathrm{size}(z_{i,j,r})=\Delta_{i,j,r}$, this enforces the transfer delay only when those very units are reused.
- **(8)** cumulative capacity: at every time point, the sum of running-activity loads and running-transfer loads does not exceed capacity $C_r$ for each resource $r$:
  $\sum_{i\in\mathcal{A}} \mathrm{pulse}(a_i,\ \mathbf{Q}_{i,r})
  \;+\;
  \sum_{i,j\in\mathcal{A}:\ \Delta_{i,j,r}>0} \mathrm{pulse}(z_{i,j,r},\ (0,\mathbf{Q}_{i,r}))
  \ \le\ C_r,\ \forall r\in\mathcal{R}.$

**Variables and Objects:**
- **(9a)** activity intervals $a_i$ (one per $i\in\mathcal{A}$) with fixed size $P_i$; their starts/ends define the schedule and the makespan via $a_{\hat{A}}$.
- **(9b)** transfer intervals $z_{i,j,r}$ (optional) for each $(i,j,r)$ with $\Delta_{i,j,r}>0$; each has size $\Delta_{i,j,r}$ and is present iff those units are actually transferred.
- **(9c)** zero-delay flows $f_{i,j,r}\in\mathbb{Z}_{\ge 0}$ for pairs with $\Delta_{i,j,r}=0$; these route resource units instantly (including from the source) and participate in the inflow/outflow balances.


| Symbol / Function | Meaning | docplex.cp reference |
|---|---|---|
| $\mathcal{A}=\{0,1,\ldots,\hat{A}\}$ | Set of activities (with $0$ = source, $\hat{A}$ = finish) | — |
| $\mathcal{R}=\{0,1,\ldots,\hat{R}\}$ | Set of primary renewable resources | — |
| $\mathcal{E}\subseteq \mathcal{A}\times\mathcal{A}$ | Precedence set: $(i,j)\in\mathcal{E}$ means $i$ precedes $j$ | — |
| $P\in\mathbb{N}^{\lvert\mathcal{A}\rvert}$ | Vector of processing times; entry $P_i$ is duration of activity $i$ | — |
| $C\in\mathbb{N}^{\lvert\mathcal{R}\rvert}$ | Vector of resource capacities; entry $C_r$ is capacity of resource $r$ | — |
| $Q\in\mathbb{Z}_{\ge 0}^{\lvert\mathcal{A}\rvert\times\lvert\mathcal{R}\rvert}$ | Consumption matrix; $Q_{i,r}$ units of resource $r$ are used by activity $i$ | — |
| $\Delta\in\mathbb{Z}_{\ge 0}^{\lvert\mathcal{A}\rvert\times\lvert\mathcal{A}\rvert\times\lvert\mathcal{R}\rvert}$ | Transfer-delay tensor; $\Delta_{i,j,r}$ time to transfer resource $r$ from $i$ to $j$ | — |
| $a_i$ | Activity interval var. for $i$ with size $P_i$ | [interval\_var](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html#docplex.cp.expression.interval_var) |
| $z_{i,j,r}$ | Optional transfer interval from $i$ to $j$ on resource $r$ with size $\Delta_{i,j,r}$ | [interval\_var (optional)](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html#docplex.cp.expression.interval_var) |
| $f_{i,j,r}\in\mathbb{Z}_{\ge 0}$ | Zero-delay flow of resource $r$ routed $i\!\to\!j$ when $\Delta_{i,j,r}=0$ | — |
| $\mathrm{end}(a_{\hat{A}})$ | End time of finish activity; the project makespan | [end\_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_of) |
| $\min\ \mathrm{end}(a_{\hat{A}})$ | Makespan minimization objective | [minimize](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.minimize) |
| $\mathrm{endBeforeStart}(a_i,a_j)$ | Enforce precedence $i\rightarrow j$ | [end\_before\_start](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_before_start) |
| $\mathrm{presenceOf}(z_{i,j,r})$ | Presence indicator (1 iff optional interval $z_{i,j,r}$ is present) | [presence\_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.presence_of) |
| $\mathrm{pulse}(a_i,\ Q_{i,r})$ | Load of $Q_{i,r}$ while $a_i$ runs (activity demand) | [pulse](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.pulse) |
| $\mathrm{pulse}(z_{i,j,r},\ (0,\ Q_{i,r}))$ | Transfer load while $z_{i,j,r}$ runs (bounded by $Q_{i,r}$) | [pulse](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.pulse) |
| $\mathrm{heightAtStart}(z,\ \mathrm{pulse}(\cdot))$ | Instantaneous load value at the start of interval $z$ (used in flow balance) | [height\_at\_start](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.height_at_start) |
| $\sum \mathrm{pulse}(\cdot)\ \le\ C_r$ (for all time) | Cumulative capacity of resource $r$ never exceeded | [always\_less\_or\_equal](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.always_less_or_equal) |
| $\sum_{j\in\mathcal{A}} f_{0,j,r}=C_r$ | Source emits exactly $C_r$ units of resource $r$ | — |


### CPLEX Implementation

#### Imports

In [ ]:
from docplex.cp.model import *
import docplex.cp.utils_visu as visu
import re
from pathlib import Path

#### Reading the data file

In [18]:
def _ints(s): 
    return list(map(int, re.findall(r"-?\d+", s)))

def _seek(lines, pat, start=0):
    p = re.compile(pat)
    for k in range(start, len(lines)):
        if p.search(lines[k]): 
            return k
    return len(lines)

def parse_rcpsp_psplib(path):
    text  = Path(path).read_text(encoding="utf-8", errors="ignore")
    lines = [ln.rstrip() for ln in text.splitlines()]
    # Basic counts
    i_jobs   = _seek(lines, r"jobs\s*\(incl\.\s*supersource/sink\s*\)\s*:\s*\d+")
    n_jobs   = (_ints(lines[i_jobs]) or [0])[0]
    i_hor    = _seek(lines, r"horizon\s*:\s*\d+", i_jobs)
    horizon  = (_ints(lines[i_hor]) or [0])[0]
    i_ren    = _seek(lines, r"-\s*renewable\s*:\s*\d+", i_hor)
    nR       = (_ints(lines[i_ren]) or [0])[0]
    # Precedence arcs
    arcs = []
    i = _seek(lines, r"PRECEDENCE\s+RELATIONS", i_ren)
    i = _seek(lines, r"successors", i) + 1
    while i < len(lines) and not lines[i].startswith("*"):
        nums = _ints(lines[i])
        if nums:
            j = nums[0] - 1
            succs = nums[3:]
            arcs += [(j, s-1) for s in succs]
        i += 1
    # Durations and demands
    durations = [0]*n_jobs
    demands   = [[0]*nR for _ in range(n_jobs)]
    i = _seek(lines, r"REQUESTS/DURATIONS", i)
    i = _seek(lines, r"-{3,}", i) + 1
    while i < len(lines) and not lines[i].startswith("*"):
        nums = _ints(lines[i])
        if len(nums) >= 3 + nR:
            j = nums[0]-1
            durations[j] = nums[2]
            demands[j]   = nums[3:3+nR]
        i += 1
    # Resource capacities
    i = _seek(lines, r"RESOURCEAVAILABILITIES", i)
    i += 1
    while i < len(lines) and not _ints(lines[i]): 
        i += 1
    capacities = (_ints(lines[i])[:nR] if i < len(lines) else [0]*nR)
    # Transfer time matrices
    transfer_times = []
    for k in range(1, max(nR,1)+1):
        i = _seek(lines, rf"TRANSFERTIMES\s+R\s+{k}\s*:", i)
        i += 1
        i = _seek(lines, r"-{5,}", i) + 1
        T = [[0]*n_jobs for _ in range(n_jobs)]
        rcount = 0
        while i < len(lines) and rcount < n_jobs:
            nums = _ints(lines[i])
            if len(nums) >= 1 + n_jobs:
                r = max(0, nums[0]-1)
                T[r] = nums[1:1+n_jobs]
                rcount += 1
            i += 1
        transfer_times.append(T)
    transfer_times = transfer_times[:nR]
    return {
        "n_jobs": n_jobs,
        "horizon": horizon,
        "n_resources": nR,
        "capacities": capacities,
        "durations": durations,
        "demands": demands,
        "precedence_arcs": arcs,
        "transfer_times": transfer_times
    }

In [28]:
filename = "../data/rcpsptt/j301_a.sm"
filepath = Path(filename)
data = parse_rcpsp_psplib(filename)

A_hat = data['n_jobs'] - 1
R_hat = data['n_resources'] - 1

A_set = range(A_hat + 1) # Set of activities {0, ..., A_hat}
R_set = range(R_hat + 1) # Set of resources {0, ..., R_hat}

P = data['durations']                           # P[i]
C = data['capacities']                          # C[r]
Q = data['demands']                             # Q[i][r]
E = data['precedence_arcs']                     # List of (i, j) tuples
Delta_matrices = data['transfer_times']         # [r][i][j]

# For easier lookup, create a (i, j, r) -> delay dictionary
Delta = {
    (i, j, r): Delta_matrices[r][i][j]
    for i in A_set
    for j in A_set
    for r in R_set
}

#### Create model and variables

In [41]:
mdl = CpoModel()

# (9a) Activity interval variables
a = [mdl.interval_var(name=f"a_{i}", size=P[i]) for i in A_set]

# (9b) Optional transfer intervals z_(i,j,r) for Delta > 0
z = {
    (i, j, r): mdl.interval_var(name=f"z_{i}_{j}_{r}", 
                              size=Delta[i, j, r], 
                              optional=True)
    for (i, j, r), delay in Delta.items() if delay > 0
}

# (9c) Zero-delay flow variables (f_i,j,r) for Delta == 0
f = {
    (i, j, r): mdl.integer_var(min=0, max=max(C) if C else 0, name=f"f_{i}_{j}_{r}")
    for (i, j, r), delay in Delta.items() if delay == 0
}

#### Solve the model

In [ ]:
print('Solving model...')
res = mdl.solve(FailLimit=100000,TimeLimit=10)
print('Solution: ')
res.print_solution()

### Aditional Resources

- **Instances for RCPSPTT**
  - https://www2.informatik.uni-osnabrueck.de/kombopt/data/rcpsp/

- **For image of the problem instance see this article:** [An efficient genetic algorithm to solve the resource-constrained project scheduling problem with transfer times](https://www.sciencedirect.com/science/article/pii/S0377221717306549)

- [Problem defintion](https://drive.google.com/file/d/1Gwfgm7mcY47d0zJWLY-SD9BVqHYsUKkP/view?usp=sharing)